In [1]:
#!pip install pyfinviz
#!pip install transformers
#!pip install torch

In [2]:
from pyfinviz.quote import Quote
import pandas as pd
import scipy
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


c:\Users\chris\OneDrive\Desktop\Fabulous\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
quote = Quote(ticker="AMZN")
df_news = quote.outer_news_df  

In [4]:
df_news


,Date,Headline,URL,From
0,01-21-2024 10:00AM,Netflix Has Its Own Tough Act to Follow,https://finance.yahoo.com/m/7a87e061-1459-316c...,(The Wall Street Journal)
1,01-21-2024 09:34AM,"Amazon Did $143 Billion in Q3 Revenue, but Her...",https://finance.yahoo.com/m/fe620463-7ee3-3827...,(Motley Fool)
2,01-21-2024 08:30AM,History Suggests the Nasdaq Will Surge in 2024...,https://finance.yahoo.com/m/47108420-ce28-3634...,(Motley Fool)
3,01-21-2024 07:36AM,3 Stocks You Can Confidently Buy After a Marke...,https://finance.yahoo.com/m/004fe837-582e-3a7e...,(Motley Fool)
4,01-21-2024 07:01AM,11 Stocks That Will Make You Rich in 5-10 Years,https://finance.yahoo.com/news/11-stocks-rich-...,(Insider Monkey)
...,...,...,...,...
95,Jan-16-24 07:30AM,"If You'd Invest $350,000 in the ""Magnificent S...",https://finance.yahoo.com/m/f0193758-0d4e-3a27...,(Motley Fool)
96,Jan-16-24 07:30AM,"If You Invest $350,000 in the ""Magnificent Sev...",https://finance.yahoo.com/m/f0193758-0d4e-3a27...,(Motley Fool)
97,Jan-16-24 06:45AM,Don't Sweat It! Layoffs Aren't a Reason to Aba...,https://investorplace.com/market360/2024/01/do...,(InvestorPlace)
98,Jan-16-24 06:15AM,1 Artificial Intelligence (AI) Stock That Has ...,https://finance.yahoo.com/m/fdb2ec58-8abf-3662...,(Motley Fool)


In [5]:

# Downloading and instantiating model
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

In [6]:
# Function to perform sentiment analysis
def get_sentiment(text):
    tokens = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        outputs = model(**tokens)
    probabilities = torch.softmax(outputs.logits, dim=1).detach().numpy()
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    sentiment_classes = ['negative', 'neutral', 'positive']
    predicted_sentiment = sentiment_classes[predicted_class]
    return predicted_sentiment, probabilities[0][0], probabilities[0][1], probabilities[0][2]

# Apply sentiment analysis to all headlines
df_news[['Sentiment', 'Negative_Probability', 'Neutral_Probability', 'Positive_Probability']] = df_news['Headline'].apply(get_sentiment).apply(pd.Series)

# Display the resulting DataFrame with sentiment predictions
df_news.head(5)

,Date,Headline,URL,From,Sentiment,Negative_Probability,Neutral_Probability,Positive_Probability
0,01-21-2024 10:00AM,Netflix Has Its Own Tough Act to Follow,https://finance.yahoo.com/m/7a87e061-1459-316c...,(The Wall Street Journal),positive,0.035119,0.082247,0.882634
1,01-21-2024 09:34AM,"Amazon Did $143 Billion in Q3 Revenue, but Her...",https://finance.yahoo.com/m/fe620463-7ee3-3827...,(Motley Fool),positive,0.032470,0.029539,0.937992
2,01-21-2024 08:30AM,History Suggests the Nasdaq Will Surge in 2024...,https://finance.yahoo.com/m/47108420-ce28-3634...,(Motley Fool),positive,0.484858,0.016098,0.499044
3,01-21-2024 07:36AM,3 Stocks You Can Confidently Buy After a Marke...,https://finance.yahoo.com/m/004fe837-582e-3a7e...,(Motley Fool),positive,0.064145,0.051028,0.884827
4,01-21-2024 07:01AM,11 Stocks That Will Make You Rich in 5-10 Years,https://finance.yahoo.com/news/11-stocks-rich-...,(Insider Monkey),positive,0.117489,0.011848,0.870663


In [7]:
print(quote.outer_ratings_df) 

         Date      Status                      Outer                  Rating  \
0   Jan-17-24  Reiterated                Wells Fargo              Overweight   
1   Jan-09-24   Initiated        BMO Capital Markets              Outperform   
2   Jan-03-24     Resumed                DA Davidson                     Buy   
3   Dec-22-23  Reiterated                    Needham                     Buy   
4   Dec-20-23     Resumed              Raymond James              Strong Buy   
5   Dec-18-23  Reiterated                   ROTH MKM                     Buy   
6   Dec-14-23  Reiterated              Piper Sandler              Overweight   
7   Nov-02-23   Initiated            HSBC Securities                     Buy   
8   Oct-24-23   Initiated  Seaport Research Partners                     Buy   
9   Oct-17-23   Initiated                     Stifel                     Buy   
10  Aug-24-23  Reiterated              Piper Sandler              Overweight   
11  Aug-23-23  Reiterated               

In [8]:
quote.fundamental_df

,Index,P/E,EPS (ttm),Insider Own,Shs Outstand,Perf Week,Market Cap,Forward P/E,EPS next Y - EPS estimate for next year,Insider Trans,...,Earnings,Payout,Avg Volume,Price,Recom,SMA20,SMA50,SMA200,Volume,Change
0,"NDX, S&P 500",81.11,1.92,12.28%,10.24B,0.10%,1605.28B,42.36,3.67,-0.03%,...,Feb 01 AMC,-,50.50M,155.34,1.17,2.34%,4.87%,18.74%,"47,867,546",1.20%


In [9]:
from pyfinviz.quote import Quote

quote = Quote(ticker="AMZN")

print(quote.exists)  # check if fetch was successful (STOCK may not exist)
print(quote.ticker)

True
AMZN


In [10]:
print(quote.fundamental_df)


          Index    P/E EPS (ttm) Insider Own Shs Outstand Perf Week  \
0  NDX, S&P 500  81.11      1.92      12.28%       10.24B     0.10%   

  Market Cap Forward P/E EPS next Y - EPS estimate for next year  \
0   1605.28B       42.36                                    3.67   

  Insider Trans  ...    Earnings Payout Avg Volume   Price Recom  SMA20  \
0        -0.03%  ...  Feb 01 AMC      -     50.50M  155.34  1.17  2.34%   

   SMA50  SMA200      Volume Change  
0  4.87%  18.74%  47,867,546  1.20%  

[1 rows x 72 columns]


In [11]:
print(quote.outer_ratings_df) 


         Date      Status                      Outer                  Rating  \
0   Jan-17-24  Reiterated                Wells Fargo              Overweight   
1   Jan-09-24   Initiated        BMO Capital Markets              Outperform   
2   Jan-03-24     Resumed                DA Davidson                     Buy   
3   Dec-22-23  Reiterated                    Needham                     Buy   
4   Dec-20-23     Resumed              Raymond James              Strong Buy   
5   Dec-18-23  Reiterated                   ROTH MKM                     Buy   
6   Dec-14-23  Reiterated              Piper Sandler              Overweight   
7   Nov-02-23   Initiated            HSBC Securities                     Buy   
8   Oct-24-23   Initiated  Seaport Research Partners                     Buy   
9   Oct-17-23   Initiated                     Stifel                     Buy   
10  Aug-24-23  Reiterated              Piper Sandler              Overweight   
11  Aug-23-23  Reiterated               

In [12]:
print(quote.exchange)  # NASD
print(quote.company_name)  # Amazon.com, Inc.
print(quote.sectors)  # ['Consumer Cyclical', 'Internet Retail', 'USA']
print(quote.fundamental_df)  # Index    P/E EPS (ttm) Insider Own  ...  SMA50  SMA200     Volume  Change
print(quote.outer_ratings_df)  # 0   Nov-04-20     Upgrade  ...                Hold → Buy  $3360 → $4000
print(quote.outer_news_df)  # 0   Jan-04-21 10:20PM  ...                   Bloomberg
print(quote.income_statement_df)  # 1      12/31/2019  ...                    22.99206
print(quote.insider_trading_df) 

NASD
Amazon.com Inc.
['Consumer Cyclical', 'Internet Retail', 'USA']
          Index    P/E EPS (ttm) Insider Own Shs Outstand Perf Week  \
0  NDX, S&P 500  81.11      1.92      12.28%       10.24B     0.10%   

  Market Cap Forward P/E EPS next Y - EPS estimate for next year  \
0   1605.28B       42.36                                    3.67   

  Insider Trans  ...    Earnings Payout Avg Volume   Price Recom  SMA20  \
0        -0.03%  ...  Feb 01 AMC      -     50.50M  155.34  1.17  2.34%   

   SMA50  SMA200      Volume Change  
0  4.87%  18.74%  47,867,546  1.20%  

[1 rows x 72 columns]
         Date      Status                      Outer                  Rating  \
0   Jan-17-24  Reiterated                Wells Fargo              Overweight   
1   Jan-09-24   Initiated        BMO Capital Markets              Outperform   
2   Jan-03-24     Resumed                DA Davidson                     Buy   
3   Dec-22-23  Reiterated                    Needham                     Buy   
4